# Пиявка - Python-based completion

Kludgier than we expected because we had to hack a somewhat reliable data source here.
---
## Setup

In [92]:
import csv
import json
import datetime

---
## World

In [93]:
rawSource = 'site-data/scraped-world.tsv'

updateWorld = dict()
today  = datetime.date.today().strftime("%m-%d-%Y")
with open(rawSource, 'r') as inputFile:
    rawData = csv.DictReader(inputFile, delimiter = '\t')
    for row in rawData:
        if 'Diamond' not in row['OTHER PLACES']:
                updateWorld[row['OTHER PLACES']] = { today: float(row['Cases'])}

if 'Queue' in updateWorld:
    del(updateWorld['Queue'])

updateWorld['US'] = updateWorld['United States']
del(updateWorld['United States'])
updateWorld['UK'] = updateWorld['United Kingdom']
del(updateWorld['United Kingdom'])
updateWorld['Denmark'] = updateWorld['Denmark*']
del(updateWorld['Denmark*'])
updateWorld['United Arab Emirates'] = updateWorld['UAE']
del(updateWorld['UAE'])
updateWorld['Bosnia and Herzegovina'] = updateWorld['Bosnia']
del(updateWorld['Bosnia'])
# UK same thing
updateWorld['US']

{'03-11-2020': 975.0}

In [101]:
with open('site-data/deaths.json', 'r') as inputFile:
    dataWorld = json.load(inputFile)

# Not in the list
dataWorld['Turkey'] = { today: 0.0, }

In [104]:
for country in updateWorld.keys():
    dataWorld[country][today] = updateWorld[country][today]

dataWorld['Denmark'][today]

264.0

---
## United States

In [115]:
rawSource = 'site-data/scraped-US.tsv'

updateUS = dict()

with open(rawSource, 'r') as inputFile:
    rawData = csv.DictReader(inputFile, delimiter = "\t")
    for row in rawData:
        updateUS[row['UNITED STATES']] = { today: float(row['Cases'])}

updateUS['Washington D.C.'] = updateUS['District of Columbia']
del(updateUS['District of Columbia'])

In [116]:
updateUS

{'Washington': {'03-11-2020': 267.0},
 'New York': {'03-11-2020': 173.0},
 'California': {'03-11-2020': 102.0},
 'Massachusetts': {'03-11-2020': 92.0},
 'Illinois': {'03-11-2020': 19.0},
 'New Jersey': {'03-11-2020': 15.0},
 'Oregon': {'03-11-2020': 15.0},
 'Florida': {'03-11-2020': 13.0},
 'Pennsylvania': {'03-11-2020': 11.0},
 'Georgia': {'03-11-2020': 11.0},
 'Colorado': {'03-11-2020': 8.0},
 'Kentucky': {'03-11-2020': 8.0},
 'Texas': {'03-11-2020': 7.0},
 'Maryland': {'03-11-2020': 6.0},
 'South Carolina': {'03-11-2020': 6.0},
 'Arizona': {'03-11-2020': 5.0},
 'North Dakota': {'03-11-2020': 5.0},
 'New Hampshire': {'03-11-2020': 4.0},
 'Nevada': {'03-11-2020': 4.0},
 'Rhode Island': {'03-11-2020': 3.0},
 'Tennessee': {'03-11-2020': 3.0},
 'Iowa': {'03-11-2020': 3.0},
 'Ohio': {'03-11-2020': 3.0},
 'Minnesota': {'03-11-2020': 3.0},
 'Wisconsin': {'03-11-2020': 3.0},
 'North Carolina': {'03-11-2020': 2.0},
 'Indiana': {'03-11-2020': 2.0},
 'Hawaii': {'03-11-2020': 2.0},
 'Vermont': {

In [117]:
with open('site-data/confirmed-US.json', 'r') as inputFile:
    dataUS = json.load(inputFile)

dataUS['Grand Princess'] = { today: 0.0, }

In [118]:
for state in updateUS.keys():
    dataUS[state][today] = updateUS[state][today]

dataUS['California']

{'01/22/2020': 0,
 '01/23/2020': 0,
 '01/24/2020': 0,
 '01/25/2020': 0,
 '01/26/2020': 2,
 '01/27/2020': 2,
 '01/28/2020': 2,
 '01/29/2020': 2,
 '01/30/2020': 2,
 '01/31/2020': 3,
 '02/01/2020': 3,
 '02/02/2020': 3,
 '02/03/2020': 6,
 '02/04/2020': 6,
 '02/05/2020': 6,
 '02/06/2020': 6,
 '02/07/2020': 6,
 '02/08/2020': 6,
 '02/09/2020': 6,
 '02/10/2020': 6,
 '02/11/2020': 7,
 '02/12/2020': 7,
 '02/13/2020': 8,
 '02/14/2020': 8,
 '02/15/2020': 8,
 '02/16/2020': 8,
 '02/17/2020': 8,
 '02/18/2020': 8,
 '02/19/2020': 8,
 '02/20/2020': 8,
 '02/21/2020': 15,
 '02/22/2020': 15,
 '02/23/2020': 15,
 '02/24/2020': 10,
 '02/25/2020': 10,
 '02/26/2020': 10,
 '02/27/2020': 11,
 '02/28/2020': 11,
 '02/29/2020': 12,
 '03/01/2020': 12,
 '03/02/2020': 21,
 '03/03/2020': 25,
 '03/04/2020': 35,
 '03/05/2020': 51,
 '03/06/2020': 59,
 '03/07/2020': 81,
 '03/08/2020': 98,
 '03/09/2020': 104,
 '03-11-2020': 102.0}

---
## US Regions


In [87]:
from covidvu.vujson import US_REGIONS_LONG

In [119]:
with open('site-data/confirmed-US.json', 'r') as inputFile:
    dataUSRegions = json.load(inputFile)

updateUSRegions = dict()

for state in updateUS:
    if US_REGIONS_LONG[state] not in updateUSRegions:
        updateUSRegions[US_REGIONS_LONG[state]] = { today: 0.0 }
    
    updateUSRegions[US_REGIONS_LONG[state]][today] += float(updateUS[state][today])

updateUSRegions

{'West': {'03-11-2020': 404.0},
 'Northeast': {'03-11-2020': 306.0},
 'Midwest': {'03-11-2020': 41.0},
 'South': {'03-11-2020': 46.0},
 'TX': {'03-11-2020': 7.0},
 'Grand Princess': {'03-11-2020': 21.0},
 'Diamond Princess': {'03-11-2020': 45.0}}

---
&#169; the COVIDvu Contributors.  All rights reserved